In [5]:
import yfinance as yf

In [6]:
sp500 = yf.Ticker("^GSPC")
data = sp500.history(period="1y")

Dow_Jones = yf.Ticker("^DJI")
data1 = Dow_Jones.history(period="1y")

Nasdaq = yf.Ticker("^NDX")
data2 = Nasdaq.history(period="1y")

data


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2024-03-22 00:00:00-04:00,5242.479980,5246.089844,5229.870117,5234.180176,3374700000,0.0,0.0
2024-03-25 00:00:00-04:00,5219.520020,5229.089844,5216.089844,5218.189941,3331360000,0.0,0.0
2024-03-26 00:00:00-04:00,5228.850098,5235.160156,5203.419922,5203.580078,3871790000,0.0,0.0
2024-03-27 00:00:00-04:00,5226.310059,5249.259766,5213.919922,5248.490234,3850500000,0.0,0.0
2024-03-28 00:00:00-04:00,5248.029785,5264.850098,5245.819824,5254.350098,3998270000,0.0,0.0
...,...,...,...,...,...,...,...
2025-03-17 00:00:00-04:00,5635.600098,5703.520020,5631.120117,5675.120117,5031770000,0.0,0.0
2025-03-18 00:00:00-04:00,5654.529785,5654.529785,5597.759766,5614.660156,4666270000,0.0,0.0
2025-03-19 00:00:00-04:00,5632.370117,5715.330078,5622.200195,5675.290039,4660090000,0.0,0.0


In [7]:
data1

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2024-03-22 00:00:00-04:00,39774.058594,39824.761719,39469.531250,39475.898438,332340000,0.0,0.0
2024-03-25 00:00:00-04:00,39410.539062,39430.171875,39296.031250,39313.640625,323670000,0.0,0.0
2024-03-26 00:00:00-04:00,39338.320312,39439.441406,39277.191406,39282.328125,306940000,0.0,0.0
2024-03-27 00:00:00-04:00,39461.980469,39769.410156,39461.980469,39760.078125,332190000,0.0,0.0
2024-03-28 00:00:00-04:00,39763.738281,39868.589844,39717.250000,39807.371094,360970000,0.0,0.0
...,...,...,...,...,...,...,...
2025-03-17 00:00:00-04:00,41460.218750,42013.101562,41412.750000,41841.628906,564190000,0.0,0.0
2025-03-18 00:00:00-04:00,41772.910156,41787.250000,41415.429688,41581.308594,569810000,0.0,0.0
2025-03-19 00:00:00-04:00,41613.191406,42178.410156,41613.191406,41964.628906,577530000,0.0,0.0


In [8]:
data2

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2024-03-22 00:00:00-04:00,18293.150391,18386.250000,18267.490234,18339.439453,4339930000,0.0,0.0
2024-03-25 00:00:00-04:00,18215.539062,18337.349609,18189.279297,18277.060547,4636940000,0.0,0.0
2024-03-26 00:00:00-04:00,18329.289062,18378.699219,18207.880859,18210.539062,5144260000,0.0,0.0
2024-03-27 00:00:00-04:00,18338.189453,18344.089844,18168.199219,18280.839844,4768930000,0.0,0.0
2024-03-28 00:00:00-04:00,18263.410156,18308.320312,18231.380859,18254.689453,5174450000,0.0,0.0
...,...,...,...,...,...,...,...
2025-03-17 00:00:00-04:00,19670.689453,19943.810547,19622.470703,19812.240234,6306340000,0.0,0.0
2025-03-18 00:00:00-04:00,19657.099609,19676.050781,19397.080078,19483.359375,6450530000,0.0,0.0
2025-03-19 00:00:00-04:00,19576.320312,19921.619141,19497.789062,19736.660156,6302310000,0.0,0.0
